# Downloading Dataset

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1mLkyvMB7EBbMpAA4f_uLBxOr34U84miD
!unzip -q retail_4_classes.zip

# Imports

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# Dataset Preparation

Note: you can also create your own dataset wrapper like in the previous lecture

https://colab.research.google.com/drive/1-uxaA8ZhpgLtctCKrzm2tF_U-_aeF_ci#scrollTo=nZld2EsoQkn9

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

preprocessing = transforms.Compose([
   transforms.Resize([224,224]),   
   transforms.RandomRotation(5),                             
   transforms.ToTensor(), 
])

trainset = ImageFolder('retail_4_classes/train', transform = preprocessing)
dataloader = DataLoader(dataset=trainset, batch_size=16, 
                        shuffle=True)

## Test reading the dataset

In [ ]:
img, label = trainset[100]

print(img.shape)
print(label)
image = np.transpose(img, [1,2,0])

plt.imshow(image)

## Test the dataset loader

In [ ]:
for i, (x,y) in enumerate(dataloader):
    print(x.shape, y.shape)
    break

In [ ]:
idx = 5
image = np.transpose(x[idx].cpu().numpy(), [1,2,0])
print(y[idx])
plt.imshow(image)

# Create Your own Model

In [ ]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_class):
        super(Net, self).__init__()

        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=7, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool3 =  nn.MaxPool2d(kernel_size=2, stride=2)

        self.classifier =  nn.Conv2d(64, n_class, kernel_size=1, stride=1, padding=0)
        
    def forward(self, x):
        relu = self.relu

        conv1 = relu(self.conv1(x))
        pool1 = self.pool1(conv1)

        conv2 = relu(self.conv2(pool1))+pool1
        pool2 = self.pool2(conv2)

        conv3 = relu(self.conv3(pool2))
        pool3 = self.pool3(conv3)

      
        global_pool = pool3.mean([2, 3], keepdim=True) # calc mean in row and cols
      
        out = self.classifier(global_pool)
        out = torch.squeeze(out)

        # in case the batch contains only single frame
        if len(out.shape)<2:
            out = torch.unsqueeze(out, 0)
            
        return out



# Testing the model

In [ ]:
model = Net(4)
pred = model(x)

print(pred.shape)
print(len(pred.shape))

# Training Utilities

In [ ]:
from torch.nn.functional import cross_entropy

def training(model, data_loader, objective_function, device, optimizer, max_iter=-1):
    model.train()

    for i, (x,y) in enumerate(data_loader):
        # moves the trainig data to device (possibly GPU)
        x = x.to(device)
        y = y.to(device)

        # inference
        logits = model(x)

        # calculate the error score
        loss = objective_function(logits, y) # menggunakan cross_entropy

        # updating the parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i==max_iter:
            break

In [ ]:

cpu = torch.device('cpu')
def count_acc(logits, label, device = cpu):
    pred = torch.argmax(logits, dim=1)
    return (pred == label).to(device).type(torch.FloatTensor).mean().item()

@torch.no_grad()
def evaluation(model, data_loader, objective_function, device, max_iter=-1):
    model.eval()

    loss_values = []
    accuracy_values = []

    for i, (x,y) in enumerate(data_loader):
        # moves the trainig data to device (possibly GPU)
        x = x.to(device)
        y = y.to(device)

        # inference
        logits = model(x)

        # calculate the error score
        loss = objective_function(logits, y)
        acc = count_acc(logits, y, device)

        loss_values.append(loss.item())
        accuracy_values.append(acc)

        if i==max_iter:
            break

    # calculate the average value of loss and accuracy
    loss = np.mean(loss_values)
    accuracy = np.mean(accuracy_values)

    return loss, accuracy

# Training

## Define the model

In [ ]:
model = Net(4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

## Data Preparation

In [ ]:
batch_size = 16

'''training data'''
trainset = ImageFolder('retail_4_classes/train', transform = preprocessing)
train_loader = DataLoader(dataset=trainset, batch_size=batch_size, 
                        shuffle=True)

'''validation data'''
val_preprocessing = transforms.Compose([
   transforms.Resize([224,224]),                            
   transforms.ToTensor(), 
])
valset = ImageFolder('retail_4_classes/val', transform = val_preprocessing)
val_loader = DataLoader(dataset=valset, batch_size=batch_size, 
                        shuffle=False)

## Training Process

In [ ]:
epochs = 10
model_checkpoint = 'best_model.pth'

best_acc = 0
for e in range(epochs):
    training(model, train_loader, cross_entropy, device, optimizer)
    loss, acc = evaluation(model, val_loader, cross_entropy, device)
    scheduler.step()

    print('epoch %d loss=%f acc=%f'%(e, loss, acc))
    
    if acc>best_acc:
        torch.save(model.state_dict(), model_checkpoint)
        best_acc = acc

# Loading the trained model and testing it

In [ ]:
model = Net(4)

model.load_state_dict(torch.load(model_checkpoint))
model.to(device)

'''validation data'''
valset = ImageFolder('retail_4_classes/val', transform = val_preprocessing)
val_loader = DataLoader(dataset=valset, batch_size=1, 
                        shuffle=False)

loss, acc = evaluation(model, val_loader, cross_entropy, device, max_iter=-1)
print('val:', loss, acc)

# Inference function

In [ ]:
@torch.no_grad()
def predict(model, x, device=cpu):
    model.eval()

    logits = model(x.to(device))
    pred = torch.argmax(logits, dim=1)
    return pred

In [ ]:
for i, (x,y) in enumerate(dataloader):
    print(x.shape, y.shape)
    break

prediction = predict(model, x, device)

print('preds:', prediction)
print('label:', y)